# Preparing Data for the Digital Exhibit

In [ ]:
import pandas as pd
import os
import re
from glob import glob
import json
import time
from tqdm.notebook import tqdm
import requests
from lxml import etree

### 1) Overview

The following notebook presents my steps in code for preparing our data for the digital exhibit. These steps include narrowing the data by 'high' probability labels, making sure the data includes the necessary entries for columns crucial for the exhibit, and pulling more data from Chronicling America's API. There were some external steps not present here–mainly, those of building a prototype of the exhibit first and then testing it on samples of our data. But suffice it to say that I first built the prototype and gleaned what aspects of the data it would need to function. Then I constructed the following pipeline to make those aspects available across as much of our data as possible.

Also, you may notice me dropping data throughout the pipeline below. This is not a disastrous oversight. Prior to running this code, I made a copy of all our data, a copy I could prepare specifically for the exhibit. This means we have a "core" dataset from our previous processes and an "exhibit" dataset which is essentially a reconstituted subset of the core data. After running all the code below, the exhibit data contains 31,257 clippings out of what was initially 51,901 'high' probability lynching reports. It also represents 2,241 Black victims out of what was initially 3,084.

### 2) Preliminary Directories and Functions

In [ ]:
# directory paths used throughout this notebook
directory = 'exhibit_prep/black_victims'
csv_paths = sorted(glob(os.path.join(directory, '*.csv')))

# function to count rows quickly. Used frequently below.
def fast_row_count(path: str) -> int:
    try:
        with open(path, 'r', encoding='utf-8', errors='ignore') as csv_file:
            rows = sum(1 for _ in csv_file)
        return max(0, rows - 1)
    except Exception:
        return len(pd.read_csv(path))
    
# scrape_carefully() function adapted again, used at two separate points below
def scrape_carefully(url, retries=3, timeout=30): # 30 second timeout b/c new Chron Am API is SLOOOOOOW - must skip pages that take longer than 30 seconds
    for attempt in range(retries):
        try:
            response = requests.get(url, timeout=timeout)
            if response.status_code == 200:
                time.sleep(3)  # set to 3 because that respects LoC's limit of 20 requests per minute. For more on LoC rate limits, visit https://www.loc.gov/apis/json-and-yaml/working-within-limits/
                return response

            if response.status_code == 429:
                time.sleep(3600) # one hour, a long wait time, but according to LoC, the time you will be banned if you get a 429 error
                continue

            # Other non-200 just sleep 3 seconds and move on
            time.sleep(3)

        except requests.RequestException:
            time.sleep(3)

    # after three retries, just move on
    return None

### 3) Update the urls to the new Chron Am format

Because Chronicling America updated to a new API and website, I needed to update all the 'url' values in our data. This was fairly simple: I just used regex to match variables in the old URLs and then put those variables in the new URL structure.

In [ ]:
def update_chron_am_urls(url):
    match = re.search(r'lccn/(sn\d+)/(\d{4}-\d{2}-\d{2})/(ed-\d+)/seq-(\d+)', url)
    sn_code, date, ed, seq = match.groups()
    return f'https://www.loc.gov/resource/{sn_code}/{date}/{ed}/?sp={seq}'

for file in tqdm(csv_paths, desc='Updating URLs'):
    df = pd.read_csv(file)
    if 'url' in df.columns:
        df['url'] = df['url'].apply(update_chron_am_urls)
        df.to_csv(file, index=False)

### 4) Drop rows that are not labelled 'high' probability

Since the exhibit is primarily concerned with presenting verified lynching reports, I subset the exhibit data by rows with 'high' probability labels. I also dropped any cases that didn't have any 'high' probability clippings.

In [ ]:
total_rows = sum(fast_row_count(path) for path in csv_paths)

with tqdm(total=total_rows, desc='Filtering rows', unit='row') as pbar:
    for path in csv_paths:
        df = pd.read_csv(path)
        pbar.update(len(df))
        
        # only keep rows with 'high' probability
        if 'probability' in df.columns:
            df = df[df['probability'] == 'high'].copy()
        
        # delete csvs with no 'high' probability clippings
        if len(df) == 0:
            os.remove(path)
        else:
            df.to_csv(path, index=False)

### 5) Builds the manifest json url

After a lot of investigation into the new Chronicling America API, I learned that I would need to call the JSON manifests for newspaper issues in order to get data to construct clipping image URLs for the exhibit. So, just as above, I built the manifest JSON URLs by matching variables in the page URL using regex. Then I put those variables into the manifest JSON URL format.

In [ ]:
def build_manifest_url(url):
    sn_code, date, ed = re.search(r'/resource/(sn\d+)/(\d{4}-\d{2}-\d{2})/(ed-\d+)', url).groups()
    return f'https://www.loc.gov/item/{sn_code}/{date}/{ed}/manifest.json'

total_rows = sum(fast_row_count(path) for path in csv_paths)

with tqdm(total=total_rows, desc='Building manifest URLs', unit='row') as pbar:
    for path in csv_paths:
        df = pd.read_csv(path)
        df['manifest'] = df['url'].apply(build_manifest_url)
        pbar.update(len(df))
        df.to_csv(path, index=False)

### 6) scrapes the manifest json for the xml url

After a lot of investigation into the new Chronicling America API, I also learned that I would need to pull the XML version of pages. The XML URLs are listed on the Manifest JSON. They contain word coordinates (i.e., the pixel location for each word as it appears on the page image). Within their URL structure, the XML URLs also have the batch file data and unique image file codes. These were yet more data I would need to construct the clipping image URLs.

To gather these XML URLs, I called the Manifest JSONs from the API. Then I matched the XML URLs with the general page URLs from our 'url' columns. When there was a match, I pulled the xml_url.

Also, just FYI: this code took days to run. The new API is slower and its rate limits are stricter, so if you're using the same process, please know it will take a long time for large amounts of data.

In [ ]:
# function for navigating and pulling xml urls from LoC's manifest json files
def pull_xml_url(manifest: dict, base_url: str):
    canvases = None
    
    # accounts for both iiif versions (v3 and v2) just in case
    if isinstance(manifest.get('items'), list):
        canvases = manifest['items']
    elif isinstance(manifest.get('sequences'), list) and manifest['sequences']:
        canvases = manifest['sequences'][0].get('canvases', [])

    if not isinstance(canvases, list):
        return None
    
    # if 'related': 'base_url', pulls the next 'seeAlso': which is the corresponding xml_url
    for canvas in canvases:
        if canvas.get('related') == base_url:
            xml_url = canvas.get('seeAlso')
            return xml_url if isinstance(xml_url, str) else None
        
    return None

total_rows = sum(fast_row_count(path) for path in csv_paths)

with tqdm(total=total_rows, desc='xml_url progress', unit='row') as pbar:
    for path in csv_paths:
        df = pd.read_csv(path)
        # adds the necessary column to hold our xml urls
        if 'xml_url' not in df.columns:
            df['xml_url'] = ''
        
        # skip rows you've already done
        for row in df.index:
            if str(df.at[row, 'xml_url']).strip():
                pbar.update(1)
                continue

            manifest_url = df.at[row, 'manifest']
            base_url = df.at[row, 'url']
            
            # pulls and puts into xml_url column
            response = scrape_carefully(manifest_url)
            if response and response.status_code == 200:
                manifest = json.loads(response.text)
                xml = pull_xml_url(manifest, base_url)
                df.at[row, 'xml_url'] = xml

            pbar.update(1)

        df.to_csv(path, index=False)

### 7) Pull line coordinate data from the xml_url pages

With the xml_urls, I was then able to pull the full text and its corresponding word coordinates for every page. I also pulled the 'HEIGHT' and 'WIDTH' values for every page. These refer to the number of pixels on each page from top to bottom, side to side.

Again, this required API requests for thousands of rows. It took several days. If you're recreating these steps, be prepared for long runtimes.

In [ ]:
def pull_xml_data(xml_url):
    # load lxml parser to more easily navigate xml pages. See documentation here: https://lxml.de/
    try:
        parser = etree.XMLParser(resolve_entities=False, recover=True)
        xml_file = etree.fromstring(xml_url, parser=parser)
    except Exception:
        return None, None, []

    # pull pg_height and pg_width from top of xml pages
    page_content = xml_file.xpath('//*[local-name()="Layout"]/*[local-name()="Page"]')
    if not page_content:
        page_content = xml_file.xpath('//*[local-name()="Page"]')
    pg_height = None
    pg_width = None
    if page_content:
        page = page_content[0]
        pg_height = page.get('HEIGHT')
        pg_width = page.get('WIDTH')

    # pulls the textline data, including line_id, WIDTH (width of line), HPOS, and VPOS, as well as the CONTENT values of each nested String ID
    # feeds that data back as lists of dictionaries (very convenient)
    text_line_data = []
    lines = xml_file.xpath('//*[local-name()="TextLine"]')
    for line in lines:
        line_id = line.get('ID') or ''
        width = line.get('WIDTH') or ''
        hpos = line.get('HPOS') or ''
        vpos = line.get('VPOS') or ''
        strings = line.xpath('./*[local-name()="String"]')
        contents = []
        for string in strings:
            word = string.get('CONTENT')
            if isinstance(word, str) and word.strip():
                contents.append(word.strip())
        content_joined = ' '.join(contents).strip()
        text_line_data.append({line_id: {'WIDTH': width, 'HPOS': hpos, 'VPOS': vpos, 'content': content_joined}})

    return pg_height, pg_width, text_line_data

total_rows = sum(fast_row_count(path) for path in csv_paths)

# off to the races – gonna take a loooong time, just so you know (days)
with tqdm(total=total_rows, desc='pulling coordinate data', unit='row') as pbar:
    for path in csv_paths:
        df = pd.read_csv(path)

        if 'pg_height' not in df.columns: df['pg_height'] = ''
        if 'pg_width'  not in df.columns: df['pg_width']  = ''
        if 'xml_content' not in df.columns: df['xml_content'] = ''

        # for to skip the rows you've already pulled - only skips if all data is pulled (pg_height, pg_width, and xml_content)
        for row in df.index:
            done_height = bool(str(df.at[row, 'pg_height']).strip())
            done_width  = bool(str(df.at[row, 'pg_width']).strip())
            done_xml_content = bool(isinstance(df.at[row, 'xml_content'], str) and df.at[row, 'xml_content'].strip())
            if done_height and done_width and done_xml_content:
                pbar.update(1)
                continue
                
            # also skip rows where xml_url is missing
            xml_url = df.at[row, 'xml_url'] if 'xml_url' in df.columns else ''
            if not (isinstance(xml_url, str) and xml_url.strip()):
                pbar.update(1)
                continue

            response = scrape_carefully(xml_url)
            if response and response.status_code == 200:
                pg_height, pg_width, xml_content = pull_xml_data(response.content)
                if pg_height: df.at[row, 'pg_height'] = pg_height
                if pg_width: df.at[row, 'pg_width']  = pg_width
                if xml_content: df.at[row, 'xml_content'] = json.dumps(xml_content, ensure_ascii=False)

            pbar.update(1)

        df.to_csv(path, index=False)

### 8) Check Results

After all these API calls, I wanted to assess how many pages I had successfully compiled. The following code is how I measured those success rates.

After two iterations of API calls and other data filtering, I had these results:

- total rows: 51901
- rows processed: 51901
- total progress: 100.00%
- xml content extracted: 50699
- nan rows: 1202
- total failed rows: 1202
- success rate: 97.68%

In [ ]:
total_rows = sum(fast_row_count(path) for path in csv_paths)

total_rows_pulled = 0
xml_count = 0
nan_count = 0
failed_count = 0

for path in csv_paths:
    df = pd.read_csv(path)

    if not {'pg_height', 'pg_width', 'xml_content'}.issubset(df.columns):
        continue

    total_rows_pulled += len(df)
    mask_xml = df['xml_content'].notna() & (df['xml_content'] != '')
    xml_count += mask_xml.sum()
    nan_count += df['xml_content'].isna().sum()
    failed_count += (~mask_xml).sum()

print(f'total rows: {total_rows}')
print(f'rows processed: {total_rows_pulled}')
print(f'total progress: {total_rows_pulled/total_rows * 100:.2f}%')
print(f'xml content extracted: {xml_count}')
print(f'nan rows: {nan_count}')
print(f'total failed rows: {failed_count}')
print(f'success rate: {xml_count/total_rows_pulled * 100:.2f}%')

# total rows: 51901
# rows processed: 51901
# total progress: 100.00%
# xml content extracted: 50699
# nan rows: 1202
# total failed rows: 1202
# success rate: 97.68%

### 9) Extract clippings from the xml_content

From the xml_content, I then pulled estimated clippings of the text around the victim's name. I did this by first identifying the text line that contained the victim's name. Then I extracted the three lines before the name and twenty lines after the victim's name.

In [ ]:
# a lil' function for parsing the xml_content correctly, avoiding errors from NaN values or empty strings
def load_xml_content(cell: str):
    try:
        data = json.loads(cell) if isinstance(cell, str) and cell.strip() else []
        return data if isinstance(data, list) else []
    except Exception:
        return []

# a lil' function for ensuring that victim names and xml_content are read as strings then lowercased (to make them comparable)
def lower_case_prep(string: str) -> str:
    return (string or '').lower()

total_rows = sum(fast_row_count(path) for path in csv_paths)

with tqdm(total=total_rows, desc='pulling xml_clippings', unit='row') as pbar:
    for path in csv_paths:
        df = pd.read_csv(path)

        if 'xml_clippings' not in df.columns:
            df['xml_clippings'] = ''
        
        # skips rows already done
        for row in df.index:
            if isinstance(df.at[row, 'xml_clippings'], str) and df.at[row, 'xml_clippings'].strip():
                pbar.update(1)
                continue

            # load the necessary data into these objects
            victim = df.at[row, 'victim']
            xml_content = df.at[row, 'xml_content'] if 'xml_content' in df.columns else ''

            if not (isinstance(victim, str) and victim.strip() and isinstance(xml_content, str) and xml_content.strip()):
                pbar.update(1)
                continue
            
            # just make sure they're strings and lowercased and read correctly using my previously defined functions
            victim_name = lower_case_prep(victim)
            xml_lines = load_xml_content(xml_content)

            # builds a list of xml content to more easily search for victim's name
            text_line_data = []
            for line in xml_lines:
                if not isinstance(line, dict) or not line:
                    continue
                line_id, line_data = next(iter(line.items()))
                content = line_data.get('content', '') if isinstance(line_data, dict) else ''
                text_line_data.append((len(text_line_data), line, lower_case_prep(content)))

            # reviews each line of xml content and identifies where the victim's name appears
            victim_name_index = None
            for index, line, content in text_line_data:
                if victim_name and victim_name in content:
                    victim_name_index = index
                    break
            
            # once name is identified in the xml, pulls:
            if victim_name_index is not None:
                # three lines of text before name (and other TextLine ID data)
                start = max(0, victim_name_index - 3)
                # 20 lines of text after name (and other TextLine ID data)
                end = min(len(text_line_data) - 1, victim_name_index + 20)
                subset = [text_line_data[j][1] for j in range(start, end + 1)]
                # and puts those new xml clippings into a new column
                df.at[row, 'xml_clippings'] = json.dumps(subset, ensure_ascii=False)

        pbar.update(1)

        df.to_csv(path, index=False)

### 10) Check Results Again

Then I checked the results again to see how much data I had prepared. I had a hunch that OCR errors in victim names would mean a fair number of missed clipping extractions. I think I was correct. The results were as follows:

- total rows: 51901
- rows processed: 51901
- total progress: 100.00%
- clippings extracted: 40425
- nan rows: 11476
- total failed rows: 11476
- success rate: 77.89%

In [ ]:
total_rows_pulled = 0
xml_count = 0
nan_count = 0
failed_count = 0

for path in csv_paths:
    df = pd.read_csv(path)

    if 'xml_clippings' not in df.columns:
        continue

    total_rows_pulled += len(df)
    mask_xml = df['xml_clippings'].notna() & (df['xml_clippings'] != '')
    xml_count += mask_xml.sum()
    nan_count += df['xml_clippings'].isna().sum()
    failed_count += (~mask_xml).sum()

print(f'total rows: {total_rows}')
print(f'rows processed: {total_rows_pulled}')
print(f'total progress: {total_rows_pulled/total_rows * 100:.2f}%')
print(f'clippings extracted: {xml_count}')
print(f'nan rows: {nan_count}')
print(f'total failed rows: {failed_count}')
print(f'success rate: {xml_count/total_rows_pulled * 100:.2f}%')

# total rows: 51901
# rows processed: 51901
# total progress: 100.00%
# clippings extracted: 40425
# nan rows: 11476
# total failed rows: 11476
# success rate: 77.89%

### 11) Build clipping_image_url

At this point, I had everything I needed to build the clipping image urls. These urls follow this pattern:

https://tile.loc.gov/image-services/iiif/service:ndnp:{submitter_code}:batch_{tarfile_name}:data:{sn_code}:{reel_no}:{date_edition_no}:{image_endpoint}/pct:{coordinate_x},{coordinate_y},{coordinate_w},coordinate_h}/!600,600/0/default.jpg

That's a base url of "https://tile.loc.gov/image-services/iiif/service:ndnp:" followed by: 
- submitter_code: extracted from xml_url
- tarfile_name: extracted from xml_url
- sn_code: extracted from xml_url
- reel_no: extracted from xml_url
- date_edition_no: extracted from xml_url
- image_endpoint: extracted from xml_url

Then the coordinates. These coordinates were calculated as follows:

- coordinate_x: lowest HPOS value in xml_clipping / pg_width
- coordinate_y: VPOS of the first line in xml_clipping / pg_height
- coordinate_w: highest WIDTH value in xml_clipping / pg_width
- coordinate_h: VPOS of the last line in xml_clipping - VPOS of the first line / pg_height

In [ ]:
# regex pattern for extracting all necessary data from the xml_url
url_pattern = re.compile(r'/service/ndnp/([^/]+)/batch_([^/]+)/data/(sn\d+)/([^/]+)/([^/]+)/([^/]+)\.xml$')

# function that takes necessary data from the xml_url
def parse_xml_url(xml_url):
    match = url_pattern.search(xml_url or '')
    if not match:
        return None
    submitter_code, tarfile_name, sn_code, reel_no, date_edition_no, image_endpoint = match.groups()
    return {'submitter_code': submitter_code, 'tarfile_name': tarfile_name, 'sn_code': sn_code, 'reel_no': reel_no, 'date_edition_no': date_edition_no, 'image_endpoint': image_endpoint}

# function for turning ints into floats (need to keep things readable)
def ensure_floats(number):
    try:
        if number is None:
            return None
        if isinstance(number, (int, float)):
            return float(number)
        string = str(number).strip().replace(',', '')
        return float(string)
    except Exception:
        return None

# little function for loading the json lists/dictionaries in xml_clipping column
def load_clipping_list(cell: str):
    try:
        data = json.loads(cell) if isinstance(cell, str) and cell.strip() else []
        return data if isinstance(data, list) else []
    except Exception:
        return []

# meaty function for pulling and building clipping_image_url
# it puts it all together from sources and calculates the coordinate values
def build_image_url(xml_url, pg_width, pg_height, clip_list: list):
    parts = parse_xml_url(xml_url)
    if not parts:
        return ''

    pg_width = ensure_floats(pg_width)
    pg_height = ensure_floats(pg_height)
    if not pg_width or not pg_height:
        return ''

    # prep the xml_clippings
    clipping_lists = []
    for entry in clip_list:
        if isinstance(entry, dict) and entry:
            _, data = next(iter(entry.items()))
            if isinstance(data, dict):
                clipping_lists.append(data)
    if not clipping_lists:
        return ''

    # assign the pertinent VPOS values - easy since they're just highest or lowest in the clipping lines
    # plus some contingencies in case they're out of order or missing
    vpos_first = ensure_floats(clipping_lists[0].get('VPOS'))
    vpos_last  = ensure_floats(clipping_lists[-1].get('VPOS'))
    if vpos_first is None or vpos_last is None:
        return ''
    if vpos_last < vpos_first:
        vpos_first, vpos_last = vpos_last, vpos_first

    # assign HPOS values - trickier since they must be combined with line WIDTH
    hpos_values   = []
    right_edges = []
    for part in clipping_lists:
        hpos = ensure_floats(part.get('HPOS'))
        width  = ensure_floats(part.get('WIDTH'))
        if hpos is not None:
            hpos_values.append(hpos)
            if width is not None:
                right_edges.append(hpos + width)
    if not hpos_values:
        return ''
    
    min_hpos = min(hpos_values)
    if right_edges:
        max_right = max(right_edges)
        width_span = max(0.0, max_right - min_hpos)
    else:
        width_max = max((ensure_floats(part.get('WIDTH')) for part in clipping_lists), default=None)
        if width_max is None:
            return ''
        width_span = max(0.0, width_max)

    height_span = max(0.0, vpos_last - vpos_first)
    if width_span == 0.0 or height_span == 0.0:
        return ''

    # the math to calculate coordinate percentages
    x_pct = round((min_hpos / pg_width) * 100.0, 1)
    y_pct = round((vpos_first / pg_height) * 100.0, 1)
    w_pct = round((width_span / pg_width) * 100.0, 1)
    h_pct = round((height_span / pg_height) * 100.0, 1)

    id_part = (f'service:ndnp:{parts["submitter_code"]}:batch_{parts["tarfile_name"]}:'f'data:{parts["sn_code"]}:{parts["reel_no"]}:{parts["date_edition_no"]}:{parts["image_endpoint"]}')
    region = f'pct:{x_pct:.2f},{y_pct:.2f},{w_pct:.2f},{h_pct:.2f}'
    return f'https://tile.loc.gov/image-services/iiif/{id_part}/{region}/!600,600/0/default.jpg'

def first_numeric(df, row, *cols):
    for c in cols:
        if c in df.columns:
            v = df.at[row, c]
            val = ensure_floats(v)
            if val is not None:
                return val
    return None

total_rows = sum(fast_row_count(path) for path in csv_paths)

with tqdm(total=total_rows, desc='build clipping_image_url', unit='row') as pbar:
    for path in csv_paths:
        df = pd.read_csv(path)

        if 'clipping_image_url' not in df.columns:
            df['clipping_image_url'] = ''

        for row in df.index:
            if isinstance(df.at[row, 'clipping_image_url'], str) and df.at[row, 'clipping_image_url'].strip():
                pbar.update(1)
                continue

            xml_url = df.at[row, 'xml_url'] if 'xml_url' in df.columns else ''
            clips_raw = df.at[row, 'xml_clippings'] if 'xml_clippings' in df.columns else ''

            if not (isinstance(xml_url, str) and xml_url.strip() and isinstance(clips_raw, str) and clips_raw.strip()):
                pbar.update(1)
                continue

            pg_width = first_numeric(df, row, 'pg_width', 'WIDTH')
            pg_height = first_numeric(df, row, 'pg_height', 'HEIGHT')
            if pg_width is None or pg_height is None:
                pbar.update(1)
                continue

            clip_list = load_clipping_list(clips_raw)
            url = build_image_url(xml_url, pg_width, pg_height, clip_list)
            if url:
                df.at[row, 'clipping_image_url'] = url

            pbar.update(1)

        df.to_csv(path, index=False)

### 12) Clean Files for Exhibit

My exhibit data had everything I needed at this point, but it also had a lot of unnecessary columns and data that would cause issues with the functionality of the exhibit. I therefore decided to strip down the data to just the columns necessary for exhibit functionality and just the rows properly prepared for presentation. I also changed some column titles since they would appear as labels in the exhibit text.

In [ ]:
import os
import pandas as pd
from glob import glob

directory = ('exhibit_prep/black_victims')
csv_paths = sorted(glob(os.path.join(directory, '*.csv')))
cols_to_drop = ['race', 'gender', 'sn_code', 'coverage', 'manifest', 'xml_url', 'pg_height', 'pg_width', 'xml_content', 'xml_clippings', 'probability', 'BERT_1', 'BERT_2', 'BERT_3', 'violence_word_count', 'racist_word_count', 'page_details']

for path in csv_paths:
    df = pd.read_csv(path)
    df = df.dropna(subset=['clipping_image_url'])
    df = df.dropna(subset=['lynch_latitude'])
    df = df.dropna(subset=['lynch_longitude'])
    pct_values = df['clipping_image_url'].str.extract(r'pct:([\d.]+),([\d.]+),([\d.]+),([\d.]+)')
    pct_values[3] = pd.to_numeric(pct_values[3], errors='coerce')
    mask = (pct_values[3].isna()) | (pct_values[3] <= 50.0)
    df = df[mask]
    df = df.drop(columns=[col for col in cols_to_drop if col in df.columns], errors= 'ignore')
    df = df.rename(columns={'reprint_date': 'Article Date', 'reprint_longitude': 'article_longitude', 'reprint_latitude': 'article_latitude', 'newspaper': 'Newspaper'})
    df['victim'] = df['victim'].astype(str).str.title()
    df['lynch_location'] = df['lynch_location'].astype(str).str.title()
    df['no_of_clippings'] = len(df)
    
    if df.empty:
        os.remove(path)
        continue
        
    df.to_csv(path, index=False)

### 13) Improve lynch_date presentation for the Exhibit

I also wanted the lynch_date data to be properly structured to appear in the exhibit text. This was annoyingly tricky. There were a number of date formats in the lynch_date columns, so I did my best to account for them all. 

In [ ]:
import re

# a function for parsing and adding suffixes to numbers
def add_date_suffix(number: int) -> str:
    if 11 <= number % 100 <= 13:
        return f'{number}th'
    return f"{number}{ {1:'st', 2:'nd', 3:'rd'}.get(number % 10, 'th') }"

# a function for converting year into 4-digit version rather than 2-digit version
# while accounting for our data range (1865-1921)
# and doing best to parse dd/mm vs. mm/dd
def format_month_date_year(text: str) -> str:
    string = str(text).strip()
    match = re.fullmatch(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})', string)
    if not match:
        return string
    a, b, yy = map(int, match.groups())
    yyyy = 1900 + yy if 0 <= yy <= 21 else 1800 + yy
    
    # if first number > 12, it's dd/mm/yyyy
    if a > 12:
        return f'{b}/{a}/{yyyy}'
    else:
        return f'{a}/{b}/{yyyy}'

def normalize_lynch_dates(df: pd.DataFrame):
    string = df['lynch_date'].astype(str).map(format_month_date_year)
    dt = pd.to_datetime(string, errors='coerce', infer_datetime_format=True)
    df['lynch_date'] = dt.apply(lambda x: f"{x.strftime('%B')} {add_date_suffix(x.day)}, {x.year}" if pd.notnull(x) else '')
    return df

for path in csv_paths:
    df = pd.read_csv(path)
    df = normalize_lynch_dates(df)
    df.to_csv(path, index=False)

### 14) Assess Final Amounts of Data for the Exhibit

At this point, I wanted to check how much data I had remaining after all these processes. I checked by total number of newspaper clippings, which turned out to be 31,257 out of our initial 51,901. I also checked by total number of victim cases represented in the data, which turned out to be 2,241 victims out of our initial 3,084. So, depending on how you measure it, our exhibit presents about 60% or 72% of our total number of cases and lynching reports.

In [ ]:
def fast_row_count(path: str) -> int:
    try:
        with open(path, 'r', encoding='utf-8', errors='ignore') as csv_file:
            rows = sum(1 for _ in csv_file)
        return max(0, rows - 1)
    except Exception:
        return len(pd.read_csv(path))

total_rows = sum(fast_row_count(path) for path in csv_paths)
total_rows

# 31,257 clippings (out of what was initially 51,901 lynching reports)

In [ ]:
remaining_csvs = glob(os.path.join(directory, '*.csv'))

print(f'Total CSVs remaining aka victims represented: {len(remaining_csvs)}')

# Total CSVs remaining aka victims represented: 2,241 (out of what was initially 3,084)

### 15) Build Maps for Exhibit

As a final step, I also created the maps to be used for the exhibit.

In [ ]:
import folium

directory = 'exhibit_prep/black_victims'
saved_directory = 'clipping_maps'
os.makedirs(saved_directory, exist_ok=True)

for file in os.listdir(directory):
    if file.endswith('.csv'):
        filepath = os.path.join(directory, file)
        try:
            df = pd.read_csv(filepath)
            coord_columns = ['lynch_latitude', 'lynch_longitude', 'article_latitude', 'article_longitude']
            df[coord_columns] = df[coord_columns].apply(pd.to_numeric, errors='coerce')

            # start location is center of continental USA
            map_center = [39.8283, -98.5795]
            folium_map = folium.Map(location=map_center, zoom_start=4, tiles="CartoDB positron")

            # circle markers for lynching sites
            for _, row in df.iterrows():
                tooltip = (
                    f"Victim: {row['victim']}<br>"
                    f"Date: {row['lynch_date']}<br>"
                    f"Location: {row['lynch_location']}"
                )
                folium.CircleMarker(
                    location=[row['lynch_latitude'], row['lynch_longitude']],
                    radius=10,
                    color='darkred',
                    fill=True,
                    fill_color='darkred',
                    fill_opacity=0.0,
                    tooltip=tooltip
                ).add_to(folium_map)

            # circle markers for article printing locations
            for _, row in df.dropna(subset=['article_latitude', 'article_longitude']).iterrows():
                url = row.get('url', '')
                url_link = f'<a href="{url}" target="_blank">Read the full page</a>' if pd.notnull(url) else ''
                popup_html = (
                    f"<b>Newspaper:</b> {row['Newspaper']}<br>"
                    f"<b>Article Date:</b> {row['Article Date']}<br>"
                    f"{url_link}<br><br>"
                    f"<b>Clipping:</b><br>{row['clippings']}"
                )

                folium.CircleMarker(
                    location=[row['article_latitude'], row['article_longitude']],
                    radius=4,
                    color='darkred',
                    fill=True,
                    fill_color='darkred',
                    fill_opacity=0.8,
                    popup=folium.Popup(popup_html, max_width=300)
                ).add_to(folium_map)

            output_path = os.path.join(saved_directory, file.replace('.csv', '.html'))
            folium_map.save(output_path)

        except Exception as e:
            print(f'Error processing {file}: {e}')